In [1]:
import os
import ipywidgets as widgets
import tensorflow as tf
from IPython import display
from dragnn.protos import spec_pb2
from dragnn.python import graph_builder
from dragnn.python import spec_builder
from dragnn.python import load_dragnn_cc_impl  # This loads the actual op definitions
from dragnn.python import render_parse_tree_graphviz
from dragnn.python import visualization
from google.protobuf import text_format
from syntaxnet import load_parser_ops  # This loads the actual op definitions
from syntaxnet import sentence_pb2
from syntaxnet.ops import gen_parser_ops
from tensorflow.python.platform import tf_logging as logging

def load_model(base_dir, master_spec_name, checkpoint_name):
    # Read the master spec
    master_spec = spec_pb2.MasterSpec()
    with open(os.path.join(base_dir, master_spec_name), "r") as f:
        text_format.Merge(f.read(), master_spec)
    spec_builder.complete_master_spec(master_spec, None, base_dir)
    logging.set_verbosity(logging.WARN)  # Turn off TensorFlow spam.

    # Initialize a graph
    graph = tf.Graph()
    with graph.as_default():
        hyperparam_config = spec_pb2.GridPoint()
        builder = graph_builder.MasterBuilder(master_spec, hyperparam_config)
        # This is the component that will annotate test sentences.
        annotator = builder.add_annotation(enable_tracing=True)
        builder.add_saver()  # "Savers" can save and load models; here, we're only going to load.

    sess = tf.Session(graph=graph)
    with graph.as_default():
        #sess.run(tf.global_variables_initializer())
        #sess.run('save/restore_all', {'save/Const:0': os.path.join(base_dir, checkpoint_name)})
        builder.saver.restore(sess, os.path.join(base_dir, checkpoint_name))
        
    def annotate_sentence(sentence):
        with graph.as_default():
            return sess.run([annotator['annotations'], annotator['traces']],
                            feed_dict={annotator['input_batch']: [sentence]})
    return annotate_sentence

segmenter_model = load_model("data/en/segmenter", "spec.textproto", "checkpoint")
parser_model = load_model("data/en", "parser_spec.textproto", "checkpoint")

In [16]:
def annotate_text(text):
    sentence = sentence_pb2.Sentence(
        text=text,
        token=[sentence_pb2.Token(word=text, start=-1, end=-1)]
    )

    # preprocess
    with tf.Session(graph=tf.Graph()) as tmp_session:
        char_input = gen_parser_ops.char_token_generator([sentence.SerializeToString()])
        preprocessed = tmp_session.run(char_input)[0]
    segmented, _ = segmenter_model(preprocessed)

    annotations, traces = parser_model(segmented[0])
    assert len(annotations) == 1
    assert len(traces) == 1
    return sentence_pb2.Sentence.FromString(annotations[0]), traces[0]
parse_tree, trace = annotate_text("John is eating pizza with a fork")  # just make sure it works
# dir(parse_tree)
parse_tree.SerializeToString()

'\x12 John is eating pizza with a fork\x1as\n\x04John\x10\x00\x18\x03 \x02*Zattribute { name: "Number" value: "Sing" } attribute { name: "fPOS" value: "PROPN++NNP" } 2\x00:\x05nsubj@\x00\x1a\x94\x02\n\x02is\x10\x05\x18\x06 \x02*\xfe\x01attribute { name: "Mood" value: "Ind" } attribute { name: "Number" value: "Sing" } attribute { name: "Person" value: "3" } attribute { name: "Tense" value: "Pres" } attribute { name: "VerbForm" value: "Fin" } attribute { name: "fPOS" value: "AUX++VBZ" } 2\x00:\x03aux@\x01\x1a\x9e\x01\n\x06eating\x10\x08\x18\r*\x85\x01attribute { name: "Tense" value: "Pres" } attribute { name: "VerbForm" value: "Part" } attribute { name: "fPOS" value: "VERB++VBG" } 2\x00:\x04root@\x01\x1ap\n\x05pizza\x10\x0f\x18\x13 \x02*Xattribute { name: "Number" value: "Sing" } attribute { name: "fPOS" value: "NOUN++NN" } 2\x00:\x03obj@\x01\x1aD\n\x04with\x10\x15\x18\x18 \x06*,attribute { name: "fPOS" value: "ADP++IN" } 2\x00:\x04case@\x01\x1a\x99\x01\n\x01a\x10\x1a\x18\x1a \x06*\x84\x

# Interactive parse tree explorer
Run the cell below, and then enter text in the interactive widget.

In [3]:
def _parse_tree_explorer():  # put stuff in a function to not pollute global scope
    text = widgets.Text("John is eating pizza with anchovies")
    # Also try: John is eating pizza with a fork
    display.display(text)
    html = widgets.HTML()
    display.display(html)

    def handle_submit(sender):
        del sender  # unused
        parse_tree, trace = annotate_text(text.value)
        html.value = u"""
        <div style="max-width: 100%">{}</div>
        <style type="text/css">svg {{ max-width: 100%; }}</style>
        """.format(render_parse_tree_graphviz.parse_tree_graph(parse_tree))

    text.on_submit(handle_submit)
_parse_tree_explorer()

# Interactive trace explorer
Run the cell below, and then enter text in the interactive widget.

In [4]:
def _trace_explorer():  # put stuff in a function to not pollute global scope
    text = widgets.Text("John is eating pizza with anchovies")
    display.display(text)

    output = visualization.InteractiveVisualization()
    display.display(display.HTML(output.initial_html()))

    def handle_submit(sender):
        del sender  # unused
        parse_tree, trace = annotate_text(text.value)
        display.display(display.HTML(output.show_trace(trace)))


    text.on_submit(handle_submit)
_trace_explorer()